Code has been adapted from the following 

Pull out Playlist information 
using the tutorial at https://www.linkedin.com/pulse/extracting-your-fav-playlist-info-spotifys-api-samantha-jones

Make Authorisiation codes 
from https://www.youtube.com/watch?v=xdq6Gz33khQ

In [1]:
!pip install requests
!pip install spotipy


[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
#Import required dependencies

import requests
import datetime
from urllib.parse import urlencode
import base64
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
from pprint import pprint
import csv
import json


In [7]:
#Spotify credentials
client_id = '********'
client_secret = '*****'

In [8]:
#Create clients to acess spotify information using tokens

class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_expire = True
    client_id = None
    client_secret = None
    token_url = "https://accounts.spotify.com/api/token"

    
    def __init__(self,client_id,client_secret, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret
        
    def get_client_credentials(self):
        """"
        returns a base64 encoded string
        
        """
        client_id = self.client_id
        client_secret =self.client_secret
        if client_secret == None or client_id == None:
            raise Exception ("You must set client_id and client_secret")
        client_creds = f"{client_id}:{client_secret}"
        client_creds_b64 = base64.b64encode(client_creds.encode())
        return client_creds_b64.decode()
    
    def get_token_headers(self):
        client_creds_b64 = self.get_client_credentials()
        return {"Authorization" : f"Basic {client_creds_b64}"}
    
    def get_token_data(self):
        return {"grant_type": "client_credentials"}
        
    def perform_auth(self):
        token_url = self.token_url
        token_data = self.get_token_data()
        token_header = self.get_token_headers()
        r = requests.post(token_url, data=token_data, headers=token_header)
        if r.status_code not in range(200,299):
            raise Exception("Could not Auth Client")
            #return False
        data = r.json()
        now = datetime.datetime.now()
        access_token = data['access_token']
        expires_in = data['expires_in']
        expires = now + datetime.timedelta(seconds=expires_in)
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now
        return True
    
    def get_access_token(self):
        token = self.access_token
        expires = self.access_token_expires
        now = datetime.datetime.now()
        if expires < now:
            self.perform_auth()
            return self.perform_auth()
        elif token == None:
            self.perform_auth()
            return self.perform_auth()        
        return token
    
    def get_resource_header(self):
        access_token = self.get_access_token()
        headers = {"Authorization" : f"Bearer {access_token}"}
        return headers
       
    
    def get_resource(self, lookup_id, resource_type='albums', version='v1'):
        endpoint = f"https://api.spotify.com/{version}/{resource_type}/{lookup_id}"
        headers = self.get_resource_header()
        r = requests.get(endpoint, headers=headers)
        if r.status_code not in range(200,299):
            return {}
        return r.json()
    
    def get_album(self, _id):
        return self.get_resource(_id, resource_type='albums')
    
    def get_artist(self, _id):
        return self.get_resource(_id, resource_type='artists')
    
    def get_track(self,_id):
        return self.get_resource(_id, resource_type='tracks')
        
    def search(self, query, search_type='artist'):
        headers = self.get_resource_header()
        endpoint  = "https://api.spotify.com/v1/search"
        data = urlencode({"q" : query , "type": search_type.lower()})
        lookup_url = f"{endpoint}?{data}"
        r=requests.get(lookup_url, headers=headers)
        print(r.status_code)
        if r.status_code not in range(200,299):
            return {}
        return r.json()

In [9]:
Spotify = SpotifyAPI(client_id,client_secret)

In [11]:
#Extract the playlists

client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

#Define the client to extract the play list 
def call_playlist(creator, playlist_id):
    
    #step1

    playlist_features_list = ["artist","album","track_name",  "track_id","danceability","energy","key","loudness","mode", "speechiness","instrumentalness","liveness","valence","tempo", "duration_ms","time_signature"]
    
    playlist_df = pd.DataFrame(columns = playlist_features_list)
    
    #step2
    
    playlist = sp.user_playlist_tracks(creator, playlist_id)["items"]
    for track in playlist:
        # Create empty dict
        playlist_features = {}
        # Get metadata
        playlist_features["artist"] = track["track"]["album"]["artists"][0]["name"]
        playlist_features["album"] = track["track"]["album"]["name"]
        playlist_features["track_name"] = track["track"]["name"]
        playlist_features["track_id"] = track["track"]["id"]
        
        # Get audio features
        audio_features = sp.audio_features(playlist_features["track_id"])[0]
        for feature in playlist_features_list[4:]:
            playlist_features[feature] = audio_features[feature]
        
        # Concat the dfs
        track_df = pd.DataFrame(playlist_features, index = [0])
        playlist_df = pd.concat([playlist_df, track_df], ignore_index = True)

    #Step 3
        
    return playlist_df

In [82]:
Top22_df = call_playlist("spotify","37i9dQZF1DX18jTM2l2fJY")
Top22_df
Top22_df.to_csv('2022Top.csv')

In [83]:
Top21_df = call_playlist("spotify","56r5qRUv3jSxADdmBkhcz7")
Top21_df
Top21_df.to_csv('2021Top.csv')

In [15]:
Top20_df = call_playlist("spotify","37i9dQZF1DX7Jl5KP2eZaS")
Top20_df
Top20_df.to_csv('2020Top.csv')

In [16]:
Top19_df = call_playlist("spotify","37i9dQZF1DWVRSukIED0e9")
Top19_df
Top19_df.to_csv('2019Top.csv')

In [17]:
Top18_df = call_playlist("spotify","37i9dQZF1DXe2bobNYDtW8")
Top18_df
Top18_df.to_csv('2018Top.csv')

In [24]:
Top17_df = call_playlist("spotify","37i9dQZF1DWTE7dVUebpUW")
Top17_df
Top17_df.to_csv('2017Top.csv')

In [84]:
Top21_df = Top21_df.reset_index()

pos = Top21_df['index'].tolist()

pos =[int(i) for i in pos]
new_pos =[x+1 for x in pos]

Top21_df['poistion'] = new_pos

Top21_df.head()

,index,artist,album,track_name,track_id,danceability,energy,key,loudness,mode,speechiness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,poistion
0,0,Harry Styles,As It Was,As It Was,4LRPiXqCikLlN15c3yImP7,0.52,0.731,6,-5.338,0,0.0557,0.00101,0.311,0.662,173.93,167303,4,1
1,1,Taylor Swift,Midnights,Anti-Hero,0V3wPSX9ygBnCm8psDIegu,0.637,0.643,4,-6.571,1,0.0519,0.000002,0.142,0.533,97.008,200690,4,2
2,2,Sam Smith,Unholy (feat. Kim Petras),Unholy (feat. Kim Petras),3nqQXoyQOWXiESFLlDF1hG,0.714,0.472,2,-7.375,1,0.0864,0.000005,0.266,0.238,131.121,156943,4,3
3,3,Jack Harlow,First Class,First Class,1rDQ4oMwGJI7B4tovsBOxc,0.905,0.563,8,-6.135,1,0.102,0.00001,0.113,0.324,106.998,173948,4,4
4,4,Kate Bush,Hounds of Love (2018 Remaster),Running Up That Hill (A Deal With God) - 2018 ...,29d0nY7TzCoi22XBqDQkiP,0.625,0.533,10,-11.903,0,0.0596,0.00266,0.0546,0.139,108.296,300840,4,5


In [26]:
track21 = Top21_df['track_id'].values.tolist()

In [27]:
tracks = track21

responses21 = []

for x in tracks:
    track_data = Spotify.get_track(x)
    responses21.append(track_data)


In [28]:
pprint(responses21[0])

{'album': {'album_type': 'single',
           'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/6KImCVD70vtIoJWnq6nGn3'},
                        'href': 'https://api.spotify.com/v1/artists/6KImCVD70vtIoJWnq6nGn3',
                        'id': '6KImCVD70vtIoJWnq6nGn3',
                        'name': 'Harry Styles',
                        'type': 'artist',
                        'uri': 'spotify:artist:6KImCVD70vtIoJWnq6nGn3'}],
           'available_markets': ['AD',
                                 'AE',
                                 'AG',
                                 'AL',
                                 'AM',
                                 'AO',
                                 'AR',
                                 'AT',
                                 'AU',
                                 'AZ',
                                 'BA',
                                 'BB',
                                 'BD',
                                 

In [29]:
song_id = []
popularity = []
art_name = []
art_id = []
explict = []

for track in responses21:
    song_id.append(track['id'])

for track in responses21:
    popularity.append(track['popularity'])

for track in responses21:
    art_name.append(track['artists'][0]['name'])

for track in responses21:
    art_id.append(track['artists'][0]['id'])

for track in responses21:
    explict.append(track['explicit'])


track21_zip = list(zip(song_id, popularity, art_name, art_id, explict))
track21_info_df =pd.DataFrame(track21_zip, columns = ["track_id", "popularity", "art_name", "art_id", "explicit"])
    
track21_info_df.head()

,track_id,popularity,art_name,art_id,explicit
0,4LRPiXqCikLlN15c3yImP7,93,Harry Styles,6KImCVD70vtIoJWnq6nGn3,False
1,02MWAaffLxlfxAUY7c5dvx,86,Glass Animals,4yvcSjfu4PC0CYQyLy4wSq,False
2,5HCyWlXZPP0y6Gqq8TgA20,87,The Kid LAROI,2tIP7SsRs7vjIcLrU85W8J,True
3,6Sq7ltF9Qa7SNFBsV5Cogx,95,Bad Bunny,4q3ewBCX7sLwd24euuV69X,True
4,1IHWl5LamUGEuP4ozKQSXZ,95,Bad Bunny,4q3ewBCX7sLwd24euuV69X,False


In [30]:
merge_21 = pd.merge(Top21_df, track21_info_df, on = 'track_id')

merge_21

,index,artist,album,track_name,track_id,danceability,energy,key,loudness,mode,...,liveness,valence,tempo,duration_ms,time_signature,poistion,popularity,art_name,art_id,explicit
0,0,Harry Styles,As It Was,As It Was,4LRPiXqCikLlN15c3yImP7,0.52,0.731,6,-5.338,0,...,0.311,0.662,173.93,167303,4,1,93,Harry Styles,6KImCVD70vtIoJWnq6nGn3,False
1,1,Glass Animals,Dreamland (+ Bonus Levels),Heat Waves,02MWAaffLxlfxAUY7c5dvx,0.761,0.525,11,-6.9,1,...,0.0921,0.531,80.87,238805,4,2,86,Glass Animals,4yvcSjfu4PC0CYQyLy4wSq,False
2,2,The Kid LAROI,STAY (with Justin Bieber),STAY (with Justin Bieber),5HCyWlXZPP0y6Gqq8TgA20,0.591,0.764,1,-5.484,1,...,0.103,0.478,169.928,141806,4,3,87,The Kid LAROI,2tIP7SsRs7vjIcLrU85W8J,True
3,3,Bad Bunny,Un Verano Sin Ti,Me Porto Bonito,6Sq7ltF9Qa7SNFBsV5Cogx,0.911,0.712,1,-5.105,0,...,0.0933,0.425,92.005,178567,4,4,95,Bad Bunny,4q3ewBCX7sLwd24euuV69X,True
4,4,Bad Bunny,Un Verano Sin Ti,Tití Me Preguntó,1IHWl5LamUGEuP4ozKQSXZ,0.65,0.715,5,-5.198,0,...,0.126,0.187,106.672,243717,4,5,95,Bad Bunny,4q3ewBCX7sLwd24euuV69X,False
5,5,Elton John,Cold Heart (PNAU Remix),Cold Heart - PNAU Remix,6zSpb8dQRaw0M1dK8PBwQz,0.796,0.798,1,-6.312,1,...,0.0952,0.942,116.032,202735,4,6,86,Elton John,3PhoLpVuITZKcymswpck5b,False
6,6,Imagine Dragons,Enemy (with JID) [from the series Arcane Leagu...,Enemy (with JID) - from the series Arcane Leag...,1r9xUipOqoNwggBpENDsvJ,0.728,0.783,11,-4.424,0,...,0.434,0.555,77.011,173381,4,7,85,Imagine Dragons,53XhwfbYqKCa1cC15pYq2q,False
7,7,Bizarrap,"Quevedo: Bzrp Music Sessions, Vol. 52","Quevedo: Bzrp Music Sessions, Vol. 52",2tTmW7RDtMQtBk7m2rYeSw,0.621,0.782,2,-5.548,1,...,0.23,0.55,128.033,198938,4,8,97,Bizarrap,716NhGYqD1jl2wI1Qkgq36,False
8,8,Bad Bunny,Un Verano Sin Ti,Ojitos Lindos,3k3NWokhRRkEPhCzPmV8TW,0.647,0.686,3,-5.745,0,...,0.528,0.268,79.928,258299,4,9,93,Bad Bunny,4q3ewBCX7sLwd24euuV69X,False
9,9,Kate Bush,Hounds of Love (2018 Remaster),Running Up That Hill (A Deal With God) - 2018 ...,29d0nY7TzCoi22XBqDQkiP,0.625,0.533,10,-11.903,0,...,0.0546,0.139,108.296,300840,4,10,81,Kate Bush,1aSxMhuvixZ8h9dK9jIDwL,False


In [31]:
artist21 = merge_21['art_id'].values.tolist()

In [32]:
artists = artist21

responses_art21 = []

for a in artists:
    artists_data = Spotify.get_artist(a)
    responses_art21.append(artists_data)

In [33]:
pprint(responses_art21[0])

{'external_urls': {'spotify': 'https://open.spotify.com/artist/6KImCVD70vtIoJWnq6nGn3'},
 'followers': {'href': None, 'total': 24543919},
 'genres': ['pop'],
 'href': 'https://api.spotify.com/v1/artists/6KImCVD70vtIoJWnq6nGn3',
 'id': '6KImCVD70vtIoJWnq6nGn3',
 'images': [{'height': 640,
             'url': 'https://i.scdn.co/image/ab6761610000e5ebf7db7c8ede90a019c54590bb',
             'width': 640},
            {'height': 320,
             'url': 'https://i.scdn.co/image/ab67616100005174f7db7c8ede90a019c54590bb',
             'width': 320},
            {'height': 160,
             'url': 'https://i.scdn.co/image/ab6761610000f178f7db7c8ede90a019c54590bb',
             'width': 160}],
 'name': 'Harry Styles',
 'popularity': 89,
 'type': 'artist',
 'uri': 'spotify:artist:6KImCVD70vtIoJWnq6nGn3'}


In [34]:
artist_id = []
followers = []
artist_name = []
genres = []


for info in responses_art21:
    artist_id.append(info['id'])

for info in responses_art21:
    followers.append(info['followers']['total'])

for info in responses_art21:
    artist_name.append(info['name'])

for info in responses_art21:
    genres.append(info['genres'])


art21_zip = list(zip(artist_id, followers, artist_name , genres,))
art21_info_df =pd.DataFrame(art21_zip, columns = ["art_id", "followers", "artist_name" , "genres"])
    
art21_info_df.head()

,art_id,followers,artist_name,genres
0,6KImCVD70vtIoJWnq6nGn3,24543919,Harry Styles,[pop]
1,4yvcSjfu4PC0CYQyLy4wSq,3493656,Glass Animals,"[gauze pop, indietronica, shiver pop]"
2,2tIP7SsRs7vjIcLrU85W8J,4096832,The Kid LAROI,"[australian hip hop, pop]"
3,4q3ewBCX7sLwd24euuV69X,59724899,Bad Bunny,"[reggaeton, trap latino]"
4,4q3ewBCX7sLwd24euuV69X,59724899,Bad Bunny,"[reggaeton, trap latino]"


In [35]:
dirt_21 = pd.merge(merge_21, art21_info_df, on = 'art_id')
dirt_21_df = dirt_21.drop_duplicates(subset=['track_name'])
dirt_21_df.head()

,index,artist,album,track_name,track_id,danceability,energy,key,loudness,mode,...,duration_ms,time_signature,poistion,popularity,art_name,art_id,explicit,followers,artist_name,genres
0,0,Harry Styles,As It Was,As It Was,4LRPiXqCikLlN15c3yImP7,0.52,0.731,6,-5.338,0,...,167303,4,1,93,Harry Styles,6KImCVD70vtIoJWnq6nGn3,False,24543919,Harry Styles,[pop]
3,42,Harry Styles,Fine Line,Watermelon Sugar,6UelLqGlWMcVH1E5c4H7lY,0.548,0.816,0,-4.209,1,...,174000,4,43,88,Harry Styles,6KImCVD70vtIoJWnq6nGn3,False,24543919,Harry Styles,[pop]
6,45,Harry Styles,Harry's House,Late Night Talking,1qEmFfgcLObUfQm0j1W2CK,0.714,0.728,10,-4.595,1,...,177955,4,46,90,Harry Styles,6KImCVD70vtIoJWnq6nGn3,False,24543919,Harry Styles,[pop]
9,1,Glass Animals,Dreamland (+ Bonus Levels),Heat Waves,02MWAaffLxlfxAUY7c5dvx,0.761,0.525,11,-6.9,1,...,238805,4,2,86,Glass Animals,4yvcSjfu4PC0CYQyLy4wSq,False,3493656,Glass Animals,"[gauze pop, indietronica, shiver pop]"
10,2,The Kid LAROI,STAY (with Justin Bieber),STAY (with Justin Bieber),5HCyWlXZPP0y6Gqq8TgA20,0.591,0.764,1,-5.484,1,...,141806,4,3,87,The Kid LAROI,2tIP7SsRs7vjIcLrU85W8J,True,4096832,The Kid LAROI,"[australian hip hop, pop]"


In [36]:
dirt_21 = pd.merge(merge_21, art21_info_df, on = 'art_id')
dirt_21_df = dirt_21.drop_duplicates(subset=['track_name'])
dirt_21_df.head()

,index,artist,album,track_name,track_id,danceability,energy,key,loudness,mode,...,duration_ms,time_signature,poistion,popularity,art_name,art_id,explicit,followers,artist_name,genres
0,0,Harry Styles,As It Was,As It Was,4LRPiXqCikLlN15c3yImP7,0.52,0.731,6,-5.338,0,...,167303,4,1,93,Harry Styles,6KImCVD70vtIoJWnq6nGn3,False,24543919,Harry Styles,[pop]
3,42,Harry Styles,Fine Line,Watermelon Sugar,6UelLqGlWMcVH1E5c4H7lY,0.548,0.816,0,-4.209,1,...,174000,4,43,88,Harry Styles,6KImCVD70vtIoJWnq6nGn3,False,24543919,Harry Styles,[pop]
6,45,Harry Styles,Harry's House,Late Night Talking,1qEmFfgcLObUfQm0j1W2CK,0.714,0.728,10,-4.595,1,...,177955,4,46,90,Harry Styles,6KImCVD70vtIoJWnq6nGn3,False,24543919,Harry Styles,[pop]
9,1,Glass Animals,Dreamland (+ Bonus Levels),Heat Waves,02MWAaffLxlfxAUY7c5dvx,0.761,0.525,11,-6.9,1,...,238805,4,2,86,Glass Animals,4yvcSjfu4PC0CYQyLy4wSq,False,3493656,Glass Animals,"[gauze pop, indietronica, shiver pop]"
10,2,The Kid LAROI,STAY (with Justin Bieber),STAY (with Justin Bieber),5HCyWlXZPP0y6Gqq8TgA20,0.591,0.764,1,-5.484,1,...,141806,4,3,87,The Kid LAROI,2tIP7SsRs7vjIcLrU85W8J,True,4096832,The Kid LAROI,"[australian hip hop, pop]"


In [37]:
dirt_21_df = dirt_21_df.sort_values(by = 'poistion')
drop_21_df = dirt_21_df.drop(['index', 'art_name', 'artist_name'], axis=1)
drop_21_df.head()

,artist,album,track_name,track_id,danceability,energy,key,loudness,mode,speechiness,...,valence,tempo,duration_ms,time_signature,poistion,popularity,art_id,explicit,followers,genres
0,Harry Styles,As It Was,As It Was,4LRPiXqCikLlN15c3yImP7,0.52,0.731,6,-5.338,0,0.0557,...,0.662,173.93,167303,4,1,93,6KImCVD70vtIoJWnq6nGn3,False,24543919,[pop]
9,Glass Animals,Dreamland (+ Bonus Levels),Heat Waves,02MWAaffLxlfxAUY7c5dvx,0.761,0.525,11,-6.9,1,0.0944,...,0.531,80.87,238805,4,2,86,4yvcSjfu4PC0CYQyLy4wSq,False,3493656,"[gauze pop, indietronica, shiver pop]"
10,The Kid LAROI,STAY (with Justin Bieber),STAY (with Justin Bieber),5HCyWlXZPP0y6Gqq8TgA20,0.591,0.764,1,-5.484,1,0.0483,...,0.478,169.928,141806,4,3,87,2tIP7SsRs7vjIcLrU85W8J,True,4096832,"[australian hip hop, pop]"
11,Bad Bunny,Un Verano Sin Ti,Me Porto Bonito,6Sq7ltF9Qa7SNFBsV5Cogx,0.911,0.712,1,-5.105,0,0.0817,...,0.425,92.005,178567,4,4,95,4q3ewBCX7sLwd24euuV69X,True,59724899,"[reggaeton, trap latino]"
19,Bad Bunny,Un Verano Sin Ti,Tití Me Preguntó,1IHWl5LamUGEuP4ozKQSXZ,0.65,0.715,5,-5.198,0,0.253,...,0.187,106.672,243717,4,5,95,4q3ewBCX7sLwd24euuV69X,False,59724899,"[reggaeton, trap latino]"


In [38]:
Order_21_df = drop_21_df[['poistion', 'track_name', 'artist', 'popularity', 'followers', 'genres', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature', 'explicit','track_id', 'art_id', 'album']]

Order_21_df = Order_21_df.reset_index()
Order_21_df = Order_21_df.drop(['index'], axis=1)

Order_21_df

,poistion,track_name,artist,popularity,followers,genres,danceability,energy,key,loudness,...,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,explicit,track_id,art_id,album
0,1,As It Was,Harry Styles,93,24543919,[pop],0.52,0.731,6,-5.338,...,0.00101,0.311,0.662,173.93,167303,4,False,4LRPiXqCikLlN15c3yImP7,6KImCVD70vtIoJWnq6nGn3,As It Was
1,2,Heat Waves,Glass Animals,86,3493656,"[gauze pop, indietronica, shiver pop]",0.761,0.525,11,-6.9,...,0.000007,0.0921,0.531,80.87,238805,4,False,02MWAaffLxlfxAUY7c5dvx,4yvcSjfu4PC0CYQyLy4wSq,Dreamland (+ Bonus Levels)
2,3,STAY (with Justin Bieber),The Kid LAROI,87,4096832,"[australian hip hop, pop]",0.591,0.764,1,-5.484,...,0,0.103,0.478,169.928,141806,4,True,5HCyWlXZPP0y6Gqq8TgA20,2tIP7SsRs7vjIcLrU85W8J,STAY (with Justin Bieber)
3,4,Me Porto Bonito,Bad Bunny,95,59724899,"[reggaeton, trap latino]",0.911,0.712,1,-5.105,...,0.000027,0.0933,0.425,92.005,178567,4,True,6Sq7ltF9Qa7SNFBsV5Cogx,4q3ewBCX7sLwd24euuV69X,Un Verano Sin Ti
4,5,Tití Me Preguntó,Bad Bunny,95,59724899,"[reggaeton, trap latino]",0.65,0.715,5,-5.198,...,0.000291,0.126,0.187,106.672,243717,4,False,1IHWl5LamUGEuP4ozKQSXZ,4q3ewBCX7sLwd24euuV69X,Un Verano Sin Ti
5,6,Cold Heart - PNAU Remix,Elton John,86,10461440,"[glam rock, mellow gold, piano rock]",0.796,0.798,1,-6.312,...,0.000042,0.0952,0.942,116.032,202735,4,False,6zSpb8dQRaw0M1dK8PBwQz,3PhoLpVuITZKcymswpck5b,Cold Heart (PNAU Remix)
6,7,Enemy (with JID) - from the series Arcane Leag...,Imagine Dragons,85,43605162,"[modern rock, rock]",0.728,0.783,11,-4.424,...,0,0.434,0.555,77.011,173381,4,False,1r9xUipOqoNwggBpENDsvJ,53XhwfbYqKCa1cC15pYq2q,Enemy (with JID) [from the series Arcane Leagu...
7,8,"Quevedo: Bzrp Music Sessions, Vol. 52",Bizarrap,97,7900141,"[argentine hip hop, pop venezolano, trap argen...",0.621,0.782,2,-5.548,...,0.033,0.23,0.55,128.033,198938,4,False,2tTmW7RDtMQtBk7m2rYeSw,716NhGYqD1jl2wI1Qkgq36,"Quevedo: Bzrp Music Sessions, Vol. 52"
8,9,Ojitos Lindos,Bad Bunny,93,59724899,"[reggaeton, trap latino]",0.647,0.686,3,-5.745,...,0.000001,0.528,0.268,79.928,258299,4,False,3k3NWokhRRkEPhCzPmV8TW,4q3ewBCX7sLwd24euuV69X,Un Verano Sin Ti
9,10,Running Up That Hill (A Deal With God) - 2018 ...,Kate Bush,81,1544689,"[art pop, art rock, baroque pop, new wave pop,...",0.625,0.533,10,-11.903,...,0.00266,0.0546,0.139,108.296,300840,4,False,29d0nY7TzCoi22XBqDQkiP,1aSxMhuvixZ8h9dK9jIDwL,Hounds of Love (2018 Remaster)


In [39]:
year21 = []

for n in range(50):
    year21.append(("2021"))

Order_21_df["year"] = year21

Order_21_df

,poistion,track_name,artist,popularity,followers,genres,danceability,energy,key,loudness,...,liveness,valence,tempo,duration_ms,time_signature,explicit,track_id,art_id,album,year
0,1,As It Was,Harry Styles,93,24543919,[pop],0.52,0.731,6,-5.338,...,0.311,0.662,173.93,167303,4,False,4LRPiXqCikLlN15c3yImP7,6KImCVD70vtIoJWnq6nGn3,As It Was,2021
1,2,Heat Waves,Glass Animals,86,3493656,"[gauze pop, indietronica, shiver pop]",0.761,0.525,11,-6.9,...,0.0921,0.531,80.87,238805,4,False,02MWAaffLxlfxAUY7c5dvx,4yvcSjfu4PC0CYQyLy4wSq,Dreamland (+ Bonus Levels),2021
2,3,STAY (with Justin Bieber),The Kid LAROI,87,4096832,"[australian hip hop, pop]",0.591,0.764,1,-5.484,...,0.103,0.478,169.928,141806,4,True,5HCyWlXZPP0y6Gqq8TgA20,2tIP7SsRs7vjIcLrU85W8J,STAY (with Justin Bieber),2021
3,4,Me Porto Bonito,Bad Bunny,95,59724899,"[reggaeton, trap latino]",0.911,0.712,1,-5.105,...,0.0933,0.425,92.005,178567,4,True,6Sq7ltF9Qa7SNFBsV5Cogx,4q3ewBCX7sLwd24euuV69X,Un Verano Sin Ti,2021
4,5,Tití Me Preguntó,Bad Bunny,95,59724899,"[reggaeton, trap latino]",0.65,0.715,5,-5.198,...,0.126,0.187,106.672,243717,4,False,1IHWl5LamUGEuP4ozKQSXZ,4q3ewBCX7sLwd24euuV69X,Un Verano Sin Ti,2021
5,6,Cold Heart - PNAU Remix,Elton John,86,10461440,"[glam rock, mellow gold, piano rock]",0.796,0.798,1,-6.312,...,0.0952,0.942,116.032,202735,4,False,6zSpb8dQRaw0M1dK8PBwQz,3PhoLpVuITZKcymswpck5b,Cold Heart (PNAU Remix),2021
6,7,Enemy (with JID) - from the series Arcane Leag...,Imagine Dragons,85,43605162,"[modern rock, rock]",0.728,0.783,11,-4.424,...,0.434,0.555,77.011,173381,4,False,1r9xUipOqoNwggBpENDsvJ,53XhwfbYqKCa1cC15pYq2q,Enemy (with JID) [from the series Arcane Leagu...,2021
7,8,"Quevedo: Bzrp Music Sessions, Vol. 52",Bizarrap,97,7900141,"[argentine hip hop, pop venezolano, trap argen...",0.621,0.782,2,-5.548,...,0.23,0.55,128.033,198938,4,False,2tTmW7RDtMQtBk7m2rYeSw,716NhGYqD1jl2wI1Qkgq36,"Quevedo: Bzrp Music Sessions, Vol. 52",2021
8,9,Ojitos Lindos,Bad Bunny,93,59724899,"[reggaeton, trap latino]",0.647,0.686,3,-5.745,...,0.528,0.268,79.928,258299,4,False,3k3NWokhRRkEPhCzPmV8TW,4q3ewBCX7sLwd24euuV69X,Un Verano Sin Ti,2021
9,10,Running Up That Hill (A Deal With God) - 2018 ...,Kate Bush,81,1544689,"[art pop, art rock, baroque pop, new wave pop,...",0.625,0.533,10,-11.903,...,0.0546,0.139,108.296,300840,4,False,29d0nY7TzCoi22XBqDQkiP,1aSxMhuvixZ8h9dK9jIDwL,Hounds of Love (2018 Remaster),2021


In [40]:
Order_21_df.to_csv('2021Year_final.csv')

In [41]:
Top20_df = Top20_df.reset_index()

pos = Top20_df['index'].tolist()

pos =[int(i) for i in pos]
new_pos =[x+1 for x in pos]

Top20_df['poistion'] = new_pos

track20 = Top20_df['track_id'].values.tolist()

In [42]:
tracks = track20

responses20 = []

for x in tracks:
    track_data = Spotify.get_track(x)
    responses20.append(track_data)

In [43]:
song_id = []
popularity = []
art_name = []
art_id = []
explict = []

for track in responses20:
    song_id.append(track['id'])

for track in responses20:
    popularity.append(track['popularity'])

for track in responses20:
    art_name.append(track['artists'][0]['name'])

for track in responses20:
    art_id.append(track['artists'][0]['id'])

for track in responses20:
    explict.append(track['explicit'])


track20_zip = list(zip(song_id, popularity, art_name, art_id, explict))
track20_info_df =pd.DataFrame(track20_zip, columns = ["track_id", "popularity", "art_name", "art_id", "explicit"])
    
merge_20 = pd.merge(Top20_df, track20_info_df, on = 'track_id')

artist20 = merge_20['art_id'].values.tolist()

In [44]:
artists = artist20

responses_art20 = []

for a in artists:
    artists_data = Spotify.get_artist(a)
    responses_art20.append(artists_data)

In [45]:
artist_id = []
followers = []
artist_name = []
genres = []


for info in responses_art20:
    artist_id.append(info['id'])

for info in responses_art20:
    followers.append(info['followers']['total'])

for info in responses_art20:
    artist_name.append(info['name'])

for info in responses_art20:
    genres.append(info['genres'])


art20_zip = list(zip(artist_id, followers, artist_name , genres,))
art20_info_df =pd.DataFrame(art20_zip, columns = ["art_id", "followers", "artist_name" , "genres"])
    
dirt_20 = pd.merge(merge_20, art20_info_df, on = 'art_id')
dirt_20_df = dirt_20.drop_duplicates(subset=['track_name'])

dirt_20_df = dirt_20_df.sort_values(by = 'poistion')
drop_20_df = dirt_20_df.drop(['index', 'art_name', 'artist_name'], axis=1)

Order_20_df = drop_20_df[['poistion', 'track_name', 'artist', 'popularity', 'followers', 'genres', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature', 'explicit','track_id', 'art_id', 'album']]

Order_20_df = Order_20_df.reset_index()
Order_20_df = Order_20_df.drop(['index'], axis=1)

year20 = []

for n in range(50):
    year20.append(("2020"))

Order_20_df["year"] = year20

Order_20_df.to_csv('2020Year_final.csv')

In [46]:
Top19_df = Top19_df.reset_index()

pos = Top19_df['index'].tolist()

pos =[int(i) for i in pos]
new_pos =[x+1 for x in pos]

Top19_df['poistion'] = new_pos

track19 = Top19_df['track_id'].values.tolist()

In [47]:
tracks = track19

responses19 = []

for x in tracks:
    track_data = Spotify.get_track(x)
    responses19.append(track_data)

In [48]:
song_id = []
popularity = []
art_name = []
art_id = []
explict = []

for track in responses19:
    song_id.append(track['id'])

for track in responses19:
    popularity.append(track['popularity'])

for track in responses19:
    art_name.append(track['artists'][0]['name'])

for track in responses19:
    art_id.append(track['artists'][0]['id'])

for track in responses19:
    explict.append(track['explicit'])


track19_zip = list(zip(song_id, popularity, art_name, art_id, explict))
track19_info_df =pd.DataFrame(track19_zip, columns = ["track_id", "popularity", "art_name", "art_id", "explicit"])
    
merge_19 = pd.merge(Top19_df, track19_info_df, on = 'track_id')

artist19 = merge_19['art_id'].values.tolist()



In [49]:
artists = artist19

responses_art19 = []

for a in artists:
    artists_data = Spotify.get_artist(a)
    responses_art19.append(artists_data)

In [50]:
artist_id = []
followers = []
artist_name = []
genres = []


for info in responses_art19:
    artist_id.append(info['id'])

for info in responses_art19:
    followers.append(info['followers']['total'])

for info in responses_art19:
    artist_name.append(info['name'])

for info in responses_art19:
    genres.append(info['genres'])


art19_zip = list(zip(artist_id, followers, artist_name , genres,))
art19_info_df =pd.DataFrame(art19_zip, columns = ["art_id", "followers", "artist_name" , "genres"])
    
dirt_19 = pd.merge(merge_19, art19_info_df, on = 'art_id')
dirt_19_df = dirt_19.drop_duplicates(subset=['track_name'])

dirt_19_df = dirt_19_df.sort_values(by = 'poistion')
drop_19_df = dirt_19_df.drop(['index', 'art_name', 'artist_name'], axis=1)

Order_19_df = drop_19_df[['poistion', 'track_name', 'artist', 'popularity', 'followers', 'genres', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature', 'explicit','track_id', 'art_id', 'album']]

Order_19_df = Order_19_df.reset_index()
Order_19_df = Order_19_df.drop(['index'], axis=1)

year19 = []

for n in range(100):
    year19.append(("2019"))

Order_19_df["year"] = year19

Order_19_df.to_csv('2019Year_final.csv')

In [51]:
Top18_df = Top18_df.reset_index()

pos = Top18_df['index'].tolist()

pos =[int(i) for i in pos]
new_pos =[x+1 for x in pos]

Top18_df['poistion'] = new_pos

track18 = Top18_df['track_id'].values.tolist()

In [52]:
tracks = track18

responses18 = []

for x in tracks:
    track_data = Spotify.get_track(x)
    responses18.append(track_data)

In [53]:
song_id = []
popularity = []
art_name = []
art_id = []
explict = []

for track in responses18:
    song_id.append(track['id'])

for track in responses18:
    popularity.append(track['popularity'])

for track in responses18:
    art_name.append(track['artists'][0]['name'])

for track in responses18:
    art_id.append(track['artists'][0]['id'])

for track in responses18:
    explict.append(track['explicit'])


track18_zip = list(zip(song_id, popularity, art_name, art_id, explict))
track18_info_df =pd.DataFrame(track18_zip, columns = ["track_id", "popularity", "art_name", "art_id", "explicit"])
    
merge_18 = pd.merge(Top18_df, track18_info_df, on = 'track_id')

artist18 = merge_18['art_id'].values.tolist()



In [54]:
artists = artist18

responses_art18 = []

for a in artists:
    artists_data = Spotify.get_artist(a)
    responses_art18.append(artists_data)

In [55]:
artist_id = []
followers = []
artist_name = []
genres = []


for info in responses_art18:
    artist_id.append(info['id'])

for info in responses_art18:
    followers.append(info['followers']['total'])

for info in responses_art18:
    artist_name.append(info['name'])

for info in responses_art18:
    genres.append(info['genres'])


art18_zip = list(zip(artist_id, followers, artist_name , genres,))
art18_info_df =pd.DataFrame(art18_zip, columns = ["art_id", "followers", "artist_name" , "genres"])
    
dirt_18 = pd.merge(merge_18, art18_info_df, on = 'art_id')
dirt_18_df = dirt_18.drop_duplicates(subset=['track_name'])

dirt_18_df = dirt_18_df.sort_values(by = 'poistion')
drop_18_df = dirt_18_df.drop(['index', 'art_name', 'artist_name'], axis=1)

Order_18_df = drop_18_df[['poistion', 'track_name', 'artist', 'popularity', 'followers', 'genres', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature', 'explicit','track_id', 'art_id', 'album']]

Order_18_df = Order_18_df.reset_index()
Order_18_df = Order_18_df.drop(['index'], axis=1)

year18 = []

for n in range(100):
    year18.append(("2018"))

Order_18_df["year"] = year18

Order_18_df.to_csv('2018Year_final.csv')

In [60]:
Top17_df = Top17_df.reset_index()

pos = Top17_df['index'].tolist()

pos =[int(i) for i in pos]
new_pos =[x+1 for x in pos]

Top17_df['poistion'] = new_pos

track17 = Top17_df['track_id'].values.tolist()

In [61]:
tracks = track17

responses17 = []

for x in tracks:
    track_data = Spotify.get_track(x)
    responses17.append(track_data)

In [62]:
song_id = []
popularity = []
art_name = []
art_id = []
explict = []

for track in responses17:
    song_id.append(track['id'])

for track in responses17:
    popularity.append(track['popularity'])

for track in responses17:
    art_name.append(track['artists'][0]['name'])

for track in responses17:
    art_id.append(track['artists'][0]['id'])

for track in responses17:
    explict.append(track['explicit'])


track17_zip = list(zip(song_id, popularity, art_name, art_id, explict))
track17_info_df =pd.DataFrame(track17_zip, columns = ["track_id", "popularity", "art_name", "art_id", "explicit"])
    
merge_17 = pd.merge(Top17_df, track17_info_df, on = 'track_id')

artist17 = merge_17['art_id'].values.tolist()



In [63]:
artists = artist17

responses_art17 = []

for a in artists:
    artists_data = Spotify.get_artist(a)
    responses_art17.append(artists_data)

In [64]:
artist_id = []
followers = []
artist_name = []
genres = []


for info in responses_art17:
    artist_id.append(info['id'])

for info in responses_art17:
    followers.append(info['followers']['total'])

for info in responses_art17:
    artist_name.append(info['name'])

for info in responses_art17:
    genres.append(info['genres'])


art17_zip = list(zip(artist_id, followers, artist_name , genres,))
art17_info_df =pd.DataFrame(art17_zip, columns = ["art_id", "followers", "artist_name" , "genres"])
    
dirt_17 = pd.merge(merge_17, art17_info_df, on = 'art_id')
dirt_17_df = dirt_17.drop_duplicates(subset=['track_name'])

dirt_17_df = dirt_17_df.sort_values(by = 'poistion')
drop_17_df = dirt_17_df.drop(['index', 'art_name', 'artist_name'], axis=1)

Order_17_df = drop_17_df[['poistion', 'track_name', 'artist', 'popularity', 'followers', 'genres', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature', 'explicit','track_id', 'art_id', 'album']]

Order_17_df = Order_17_df.reset_index()
Order_17_df = Order_17_df.drop(['index'], axis=1)

year17 = []

for n in range(100):
    year17.append(("2017"))

Order_17_df["year"] = year17

Order_17_df.to_csv('2017Year_final.csv')

In [72]:
Top22_df = Top22_df.reset_index()

pos = Top22_df['index'].tolist()

pos =[int(i) for i in pos]
new_pos =[x+1 for x in pos]

Top22_df['poistion'] = new_pos

track22 = Top22_df['track_id'].values.tolist()

In [73]:
tracks = track22

responses22 = []

for x in tracks:
    track_data = Spotify.get_track(x)
    responses22.append(track_data)

In [74]:
song_id = []
popularity = []
art_name = []
art_id = []
explict = []

for track in responses22:
    song_id.append(track['id'])

for track in responses22:
    popularity.append(track['popularity'])

for track in responses22:
    art_name.append(track['artists'][0]['name'])

for track in responses22:
    art_id.append(track['artists'][0]['id'])

for track in responses22:
    explict.append(track['explicit'])


track22_zip = list(zip(song_id, popularity, art_name, art_id, explict))
track22_info_df =pd.DataFrame(track22_zip, columns = ["track_id", "popularity", "art_name", "art_id", "explicit"])
    
merge_22 = pd.merge(Top22_df, track22_info_df, on = 'track_id')

artist22 = merge_22['art_id'].values.tolist()



In [75]:
artists = artist22

responses_art22 = []

for a in artists:
    artists_data = Spotify.get_artist(a)
    responses_art22.append(artists_data)

In [79]:
artist_id = []
followers = []
artist_name = []
genres = []


for info in responses_art22:
    artist_id.append(info['id'])

for info in responses_art22:
    followers.append(info['followers']['total'])

for info in responses_art22:
    artist_name.append(info['name'])

for info in responses_art22:
    genres.append(info['genres'])


art22_zip = list(zip(artist_id, followers, artist_name , genres,))
art22_info_df =pd.DataFrame(art22_zip, columns = ["art_id", "followers", "artist_name" , "genres"])
    
dirt_22 = pd.merge(merge_22, art22_info_df, on = 'art_id')
dirt_22_df = dirt_22.drop_duplicates(subset=['track_name'])

dirt_22_df = dirt_22_df.sort_values(by = 'poistion')
drop_22_df = dirt_22_df.drop(['index', 'art_name', 'artist_name'], axis=1)

Order_22_df = drop_22_df[['poistion', 'track_name', 'artist', 'popularity', 'followers', 'genres', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature', 'explicit','track_id', 'art_id', 'album']]

Order_22_df = Order_22_df.reset_index()
Order_22_df = Order_22_df.drop(['index'], axis=1)

year22 = []

for n in range(93):
    year22.append(("2022"))

Order_22_df["year"] = year22

Order_22_df.to_csv('2022Year_final.csv')

In [88]:
Top16_df = call_playlist("spotify","37i9dQZF1DX8XZ6AUo9R4R")
Top16_df
Top16_df.to_csv('2016Top.csv')

Top16_df = Top16_df.reset_index()

pos = Top16_df['index'].tolist()

pos =[int(i) for i in pos]
new_pos =[x+1 for x in pos]

Top16_df['poistion'] = new_pos

track16 = Top16_df['track_id'].values.tolist()

tracks = track16

responses16 = []

In [89]:
for x in tracks:
    track_data = Spotify.get_track(x)
    responses16.append(track_data)
    
song_id = []
popularity = []
art_name = []
art_id = []
explict = []

for track in responses16:
    song_id.append(track['id'])

for track in responses16:
    popularity.append(track['popularity'])

for track in responses16:
    art_name.append(track['artists'][0]['name'])

for track in responses16:
    art_id.append(track['artists'][0]['id'])

for track in responses16:
    explict.append(track['explicit'])


track16_zip = list(zip(song_id, popularity, art_name, art_id, explict))
track16_info_df =pd.DataFrame(track16_zip, columns = ["track_id", "popularity", "art_name", "art_id", "explicit"])
    
merge_16 = pd.merge(Top16_df, track16_info_df, on = 'track_id')

artist16 = merge_16['art_id'].values.tolist()

In [91]:
artists = artist16

responses_art16 = []

for a in artists:
    artists_data = Spotify.get_artist(a)
    responses_art16.append(artists_data)

artist_id = []
followers = []
artist_name = []
genres = []


for info in responses_art16:
    artist_id.append(info['id'])

for info in responses_art16:
    followers.append(info['followers']['total'])

for info in responses_art16:
    artist_name.append(info['name'])

for info in responses_art16:
    genres.append(info['genres'])


art16_zip = list(zip(artist_id, followers, artist_name , genres,))
art16_info_df =pd.DataFrame(art16_zip, columns = ["art_id", "followers", "artist_name" , "genres"])
    
dirt_16 = pd.merge(merge_16, art16_info_df, on = 'art_id')
dirt_16_df = dirt_16.drop_duplicates(subset=['track_name'])

dirt_16_df = dirt_16_df.sort_values(by = 'poistion')
drop_16_df = dirt_16_df.drop(['index', 'art_name', 'artist_name'], axis=1)

Order_16_df = drop_16_df[['poistion', 'track_name', 'artist', 'popularity', 'followers', 'genres', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature', 'explicit','track_id', 'art_id', 'album']]

Order_16_df = Order_16_df.reset_index()
Order_16_df = Order_16_df.drop(['index'], axis=1)

year16 = []

for n in range(99):
    year16.append(("2016"))

Order_16_df["year"] = year16

Order_16_df.to_csv('2016Year_final.csv')

In [101]:
Top15_df = call_playlist("spotify","37i9dQZF1DX9ukdrXQLJGZ")
Top15_df
Top15_df.to_csv('2015Top.csv')

Top15_df = Top15_df.reset_index()

pos = Top15_df['index'].tolist()

pos =[int(i) for i in pos]
new_pos =[x+1 for x in pos]

Top15_df['poistion'] = new_pos

track15 = Top15_df['track_id'].values.tolist()

tracks = track15

responses15 = []

In [102]:
for x in tracks:
    track_data = Spotify.get_track(x)
    responses15.append(track_data)
    
song_id = []
popularity = []
art_name = []
art_id = []
explict = []

for track in responses15:
    song_id.append(track['id'])

for track in responses15:
    popularity.append(track['popularity'])

for track in responses15:
    art_name.append(track['artists'][0]['name'])

for track in responses15:
    art_id.append(track['artists'][0]['id'])

for track in responses15:
    explict.append(track['explicit'])


track15_zip = list(zip(song_id, popularity, art_name, art_id, explict))
track15_info_df =pd.DataFrame(track15_zip, columns = ["track_id", "popularity", "art_name", "art_id", "explicit"])
    
merge_15 = pd.merge(Top15_df, track15_info_df, on = 'track_id')

artist15 = merge_15['art_id'].values.tolist()

In [104]:
artists = artist15

responses_art15 = []

for a in artists:
    artists_data = Spotify.get_artist(a)
    responses_art15.append(artists_data)

artist_id = []
followers = []
artist_name = []
genres = []


for info in responses_art15:
    artist_id.append(info['id'])

for info in responses_art15:
    followers.append(info['followers']['total'])

for info in responses_art15:
    artist_name.append(info['name'])

for info in responses_art15:
    genres.append(info['genres'])


art15_zip = list(zip(artist_id, followers, artist_name , genres,))
art15_info_df =pd.DataFrame(art15_zip, columns = ["art_id", "followers", "artist_name" , "genres"])
    
dirt_15 = pd.merge(merge_15, art15_info_df, on = 'art_id')
dirt_15_df = dirt_15.drop_duplicates(subset=['track_name'])

dirt_15_df = dirt_15_df.sort_values(by = 'poistion')
drop_15_df = dirt_15_df.drop(['index', 'art_name', 'artist_name'], axis=1)

Order_15_df = drop_15_df[['poistion', 'track_name', 'artist', 'popularity', 'followers', 'genres', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature', 'explicit','track_id', 'art_id', 'album']]

Order_15_df = Order_15_df.reset_index()
Order_15_df = Order_15_df.drop(['index'], axis=1)

year15 = []

for n in range(100):
    year15.append(("2015"))

Order_15_df["year"] = year15

Order_15_df.to_csv('2015Year_final.csv')

In [105]:
Top14_df = call_playlist("spotify","37i9dQZF1DX0h0QnLkMBl4")
Top14_df
Top14_df.to_csv('2014Top.csv')

Top14_df = Top14_df.reset_index()

pos = Top14_df['index'].tolist()

pos =[int(i) for i in pos]
new_pos =[x+1 for x in pos]

Top14_df['poistion'] = new_pos

track14 = Top14_df['track_id'].values.tolist()

tracks = track14

responses14 = []


In [106]:
for x in tracks:
    track_data = Spotify.get_track(x)
    responses14.append(track_data)
    
song_id = []
popularity = []
art_name = []
art_id = []
explict = []

for track in responses14:
    song_id.append(track['id'])

for track in responses14:
    popularity.append(track['popularity'])

for track in responses14:
    art_name.append(track['artists'][0]['name'])

for track in responses14:
    art_id.append(track['artists'][0]['id'])

for track in responses14:
    explict.append(track['explicit'])


track14_zip = list(zip(song_id, popularity, art_name, art_id, explict))
track14_info_df =pd.DataFrame(track14_zip, columns = ["track_id", "popularity", "art_name", "art_id", "explicit"])
    
merge_14 = pd.merge(Top14_df, track14_info_df, on = 'track_id')

artist14 = merge_14['art_id'].values.tolist()


In [107]:
artists = artist14

responses_art14 = []

for a in artists:
    artists_data = Spotify.get_artist(a)
    responses_art14.append(artists_data)

artist_id = []
followers = []
artist_name = []
genres = []


for info in responses_art14:
    artist_id.append(info['id'])

for info in responses_art14:
    followers.append(info['followers']['total'])

for info in responses_art14:
    artist_name.append(info['name'])

for info in responses_art14:
    genres.append(info['genres'])


art14_zip = list(zip(artist_id, followers, artist_name , genres,))
art14_info_df =pd.DataFrame(art14_zip, columns = ["art_id", "followers", "artist_name" , "genres"])
    
dirt_14 = pd.merge(merge_14, art14_info_df, on = 'art_id')
dirt_14_df = dirt_14.drop_duplicates(subset=['track_name'])

dirt_14_df = dirt_14_df.sort_values(by = 'poistion')
drop_14_df = dirt_14_df.drop(['index', 'art_name', 'artist_name'], axis=1)

Order_14_df = drop_14_df[['poistion', 'track_name', 'artist', 'popularity', 'followers', 'genres', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature', 'explicit','track_id', 'art_id', 'album']]

Order_14_df = Order_14_df.reset_index()
Order_14_df = Order_14_df.drop(['index'], axis=1)

year14 = []

for n in range(100):
    year14.append(("2014"))

Order_14_df["year"] = year14

Order_14_df.to_csv('2014Year_final.csv')


In [108]:
Top13_df = call_playlist("spotify","37i9dQZF1DX3Sp0P28SIer")
Top13_df
Top13_df.to_csv('2013Top.csv')

Top13_df = Top13_df.reset_index()

pos = Top13_df['index'].tolist()

pos =[int(i) for i in pos]
new_pos =[x+1 for x in pos]

Top13_df['poistion'] = new_pos

track13 = Top13_df['track_id'].values.tolist()

tracks = track13

responses13 = []


In [109]:
for x in tracks:
    track_data = Spotify.get_track(x)
    responses13.append(track_data)
    
song_id = []
popularity = []
art_name = []
art_id = []
explict = []

for track in responses13:
    song_id.append(track['id'])

for track in responses13:
    popularity.append(track['popularity'])

for track in responses13:
    art_name.append(track['artists'][0]['name'])

for track in responses13:
    art_id.append(track['artists'][0]['id'])

for track in responses13:
    explict.append(track['explicit'])


track13_zip = list(zip(song_id, popularity, art_name, art_id, explict))
track13_info_df =pd.DataFrame(track13_zip, columns = ["track_id", "popularity", "art_name", "art_id", "explicit"])
    
merge_13 = pd.merge(Top13_df, track13_info_df, on = 'track_id')

artist13 = merge_13['art_id'].values.tolist()


In [110]:
artists = artist13

responses_art13 = []

for a in artists:
    artists_data = Spotify.get_artist(a)
    responses_art13.append(artists_data)

artist_id = []
followers = []
artist_name = []
genres = []


for info in responses_art13:
    artist_id.append(info['id'])

for info in responses_art13:
    followers.append(info['followers']['total'])

for info in responses_art13:
    artist_name.append(info['name'])

for info in responses_art13:
    genres.append(info['genres'])


art13_zip = list(zip(artist_id, followers, artist_name , genres,))
art13_info_df =pd.DataFrame(art13_zip, columns = ["art_id", "followers", "artist_name" , "genres"])
    
dirt_13 = pd.merge(merge_13, art13_info_df, on = 'art_id')
dirt_13_df = dirt_13.drop_duplicates(subset=['track_name'])

dirt_13_df = dirt_13_df.sort_values(by = 'poistion')
drop_13_df = dirt_13_df.drop(['index', 'art_name', 'artist_name'], axis=1)

Order_13_df = drop_13_df[['poistion', 'track_name', 'artist', 'popularity', 'followers', 'genres', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature', 'explicit','track_id', 'art_id', 'album']]

Order_13_df = Order_13_df.reset_index()
Order_13_df = Order_13_df.drop(['index'], axis=1)

year13 = []

for n in range(100):
    year13.append(("2013"))

Order_13_df["year"] = year13

Order_13_df.to_csv('2013Year_final.csv')


In [111]:
Top12_df = call_playlist("spotify","37i9dQZF1DX0yEZaMOXna3")
Top12_df
Top12_df.to_csv('2012Top.csv')

Top12_df = Top12_df.reset_index()

pos = Top12_df['index'].tolist()

pos =[int(i) for i in pos]
new_pos =[x+1 for x in pos]

Top12_df['poistion'] = new_pos

track12 = Top12_df['track_id'].values.tolist()

tracks = track12

responses12 = []

In [112]:
for x in tracks:
    track_data = Spotify.get_track(x)
    responses12.append(track_data)
    
song_id = []
popularity = []
art_name = []
art_id = []
explict = []

for track in responses12:
    song_id.append(track['id'])

for track in responses12:
    popularity.append(track['popularity'])

for track in responses12:
    art_name.append(track['artists'][0]['name'])

for track in responses12:
    art_id.append(track['artists'][0]['id'])

for track in responses12:
    explict.append(track['explicit'])


track12_zip = list(zip(song_id, popularity, art_name, art_id, explict))
track12_info_df =pd.DataFrame(track12_zip, columns = ["track_id", "popularity", "art_name", "art_id", "explicit"])
    
merge_12 = pd.merge(Top12_df, track12_info_df, on = 'track_id')

artist12 = merge_12['art_id'].values.tolist()


In [113]:
artists = artist12

responses_art12 = []

for a in artists:
    artists_data = Spotify.get_artist(a)
    responses_art12.append(artists_data)

artist_id = []
followers = []
artist_name = []
genres = []


for info in responses_art12:
    artist_id.append(info['id'])

for info in responses_art12:
    followers.append(info['followers']['total'])

for info in responses_art12:
    artist_name.append(info['name'])

for info in responses_art12:
    genres.append(info['genres'])


art12_zip = list(zip(artist_id, followers, artist_name , genres,))
art12_info_df =pd.DataFrame(art12_zip, columns = ["art_id", "followers", "artist_name" , "genres"])
    
dirt_12 = pd.merge(merge_12, art12_info_df, on = 'art_id')
dirt_12_df = dirt_12.drop_duplicates(subset=['track_name'])

dirt_12_df = dirt_12_df.sort_values(by = 'poistion')
drop_12_df = dirt_12_df.drop(['index', 'art_name', 'artist_name'], axis=1)

Order_12_df = drop_12_df[['poistion', 'track_name', 'artist', 'popularity', 'followers', 'genres', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature', 'explicit','track_id', 'art_id', 'album']]

Order_12_df = Order_12_df.reset_index()
Order_12_df = Order_12_df.drop(['index'], axis=1)

year12 = []

for n in range(100):
    year12.append(("2012"))

Order_12_df["year"] = year12

Order_12_df.to_csv('2012Year_final.csv')
